In [1]:
# !pip install -q python-docx feedparser bs4



```
important libraries
```



In [2]:
from urllib import request
import numpy as np
import pandas as pd
from nltk import word_tokenize, line_tokenize, sent_tokenize
import nltk
from nltk.corpus import stopwords
import itertools
import operator
#nltk.download()
stop_words = set(stopwords.words('english'))
from nltk.util import bigrams, ngrams

### Saving function

In [3]:
#funciton to save web data to text
def save_to_txt(filename, data):
   with open(filename, 'w', encoding="utf-8") as f:
    f.write(data)

### Extract data from link

In [4]:
# import feedparser
# from bs4 import BeautifulSoup
# #WEB SOURCE 1
# llog = feedparser.parse("http://languagelog.ldc.upenn.edu/nll/?feed=atom")
# entries = llog.get('entries')
# all_content_1 = ''
# for entry in entries:
#   for content in entry.get('content'):
#     all_content_1+="<br>"+content.value

# raw_text_from_web = BeautifulSoup(all_content_1, 'html.parser').get_text()

# raw_text_from_web_filepath = 'raw_text_from_web.txt'
# save_to_txt(raw_text_from_web_filepath, raw_text_from_web)

# total_sents_in_web = sent_tokenize(raw_text_from_web)


In [27]:
import feedparser
from bs4 import BeautifulSoup
#WEB SOURCE 1
llog = feedparser.parse("http://languagelog.ldc.upenn.edu/nll/?feed=atom")
entries = llog.get('entries')
all_content_1 = ''
for entry in entries:
  for content in entry.get('content'):
    all_content_1+="<br>"+content.value

fireside = feedparser.parse("https://feeds.fireside.fm/bibleinayear/rss")

all_content_2 = ''
for entry in fireside.get('entries'):
  for content in entry.get('content'):
    all_content_2+="<br>"+content.value



raw_text_from_web = BeautifulSoup(all_content_1+"<br>"+all_content_2, 'html.parser').get_text()
raw_text_from_web_filepath = 'raw_text_from_web.txt'
save_to_txt(raw_text_from_web_filepath, raw_text_from_web)

total_sents_in_web = sent_tokenize(raw_text_from_web)

### Create a wordlist and sentence wordlist

In [28]:
from nltk.corpus.reader import WordListCorpusReader

#creating wrodlist
word_list_corpus = WordListCorpusReader('.', [raw_text_from_web_filepath])

# geting the text from wordlist
wordlist_text = word_list_corpus.raw()


#Find the words in a string using tokenizers
wordlist_text_token = word_tokenize(wordlist_text)

#Find the sentences in a string using tokenizers
total_sents_worldlist = sent_tokenize(wordlist_text)

### Shift one row and zip to get bigrams

In [29]:
#here we are getting all bigrams of our sentences 
worldlist_bigrams = [e for x in total_sents_worldlist for e in zip(x.split(" ")[:-1], x.split(" ")[1:])]

#here we calculate bigrams frequency
frequienceB = nltk.FreqDist(worldlist_bigrams)

In [30]:
frequienceB

FreqDist({('the', 'complete'): 553, ('complete', 'reading'): 553, ('For', 'the'): 549, ('visit', 'ascensionpress.com/bibleinayear.'): 538, ('reading', 'plan,'): 531, ('not', 'be'): 528, ('may', 'not'): 522, ('suitable', 'for'): 521, ('that', 'may'): 520, ('adult', 'themes'): 519, ...})

### Get Unigrams and their Counts

In [31]:
#here we are getting all bigrams of our sentences 
worldlist_unigram = [e for x in total_sents_worldlist for e in zip(x.split())]

#here we calculate unigram frequency
frequienceU = nltk.FreqDist(worldlist_unigram)
frequienceU

FreqDist({('the',): 2883, ('and',): 1642, ('of',): 1493, ('to',): 1058, ('that',): 888, ('is',): 842, ('for',): 782, ('in',): 675, ('be',): 660, ('reading',): 651, ...})

### Creating bigram count Table

Defining a function which will create a bigram count table for a sentance in the corpus.

#### Helper functions

In [32]:
# Function 1
# Creating a function which will return all possible pairs  from word list
# i.e if we pass ['apple','banna']
# will return [('apple','apple'),('apple','banna'),('banna','apple'),('banna','banna')]

def all_pairs(wordlist):
    diff_pairs_list = list(itertools.permutations(wordlist,2))
    same_pairs_list = []
    for word in wordlist:
        same_pair = (word,word)
        same_pairs_list.append(same_pair)
    all_pairs_list = diff_pairs_list + same_pairs_list
    return all_pairs_list
    

In [33]:
def bigram_count_table(sent):
    words = word_tokenize(sent)
    # removing smilar words
    vocab = np.unique(words)
    vocab_index = {word: i for i, word in enumerate(vocab)}
 
    # Create bigrams from all words in corpus
#     bi_grams = list(set(bigrams(words)))
    all_pairs_list = all_pairs(vocab)
 
    
    # initializing a zero metrix of size (number of unique words in the sentance)
    zeros_matrix = np.zeros((len(vocab), len(vocab)),dtype=int)
 
    # Loop through the bigrams taking the current and previous word,
    # and the number of occurrences of the bigram.
    for pair in all_pairs_list:
        current = pair[1]
        previous = pair[0]
        
        pos_current = vocab_index[current]
        pos_previous = vocab_index[previous]
        #the number of occurrences of the bigram in corpus.
        pair_freq = frequienceB.get(pair)
        
        if pair_freq is None:
            pair_freq = 0
        zeros_matrix[pos_current][pos_previous] = pair_freq
        
    prob_matrix = np.matrix(zeros_matrix)
    # return the matrix and the index
    table = pd.DataFrame(prob_matrix,columns=vocab_index,index = vocab_index).T
    return table

In [13]:
sent = 'this is test of the the the'
bigram_count_table(sent)

,is,of,test,the,this
is,0,0,0,28,2
of,0,0,0,319,18
test,0,0,0,1,0
the,0,0,0,1,0
this,5,0,0,0,0


### Bigram probability table

This table is created for the purpose that, if we want we can create sentance probability from this table easily

In [14]:
def bigram_prob_table(sent):
   # tokanizing the sent
    words = word_tokenize(sent)
    # removing repeted words
    vocab = np.unique(words)
    # Creating a dictionary which contain the index of the word
    vocab_index = {word: i for i, word in enumerate(vocab)}
 
    # Create bigrams from all the words in the sentance
    all_pairs_list = all_pairs(vocab)
 
    
    # initializing a metrix of size number of words in the sentance
    zeros_matrix = np.zeros((len(vocab), len(vocab)))
 
    # Loop through the bigrams taking the current and previous word,
    for pair in all_pairs_list:
        current = pair[1]
        previous = pair[0]
        
        # Extracting position of the current and previous words from vocab_index
        pos_current = vocab_index[current]
        pos_previous = vocab_index[previous]
        #Converting previous word into unigram to get its frequncy in the corpus from frequienceU
        unigram = tuple(previous.split())
        
        # the number of occurrences of the bigram.
        pair_freq = frequienceB.get(pair)
        unigram_freq = frequienceU.get(unigram)
        if pair_freq is None:
            pair_freq = 0
        if unigram_freq is None:
            unigram_freq = 1 
#         prob = round(bigram_freq /unigram_freq,4)
        prob = pair_freq /unigram_freq
        zeros_matrix[pos_current][pos_previous] = prob
        
    prob_matrix = np.matrix(zeros_matrix)
    # return the matrix and the index
    table = pd.DataFrame(prob_matrix,columns=vocab_index,index = vocab_index).T
    return table

In [15]:
sent

'this is test of the the the'

In [16]:
bigram_prob_table(sent)

,is,of,test,the,this
is,0.000000,0.0,0.0,0.033254,0.002375
of,0.000000,0.0,0.0,0.213664,0.012056
test,0.000000,0.0,0.0,0.142857,0.000000
the,0.000000,0.0,0.0,0.000347,0.000000
this,0.042017,0.0,0.0,0.000000,0.000000


### Calculating Sentance probability

In [17]:
# Creating a function to calculate the probablity of a sentance in given corpus using bigram count table
def calculate_sent_prob(sentance):
    # Creating table uning the defined function
    table = bigram_count_table(sentance)
    
    # Converting the sentance into tokens
    wordlist_tokens = word_tokenize(sentance)
    
    # Creating bigrams of the sentance
    wordlist_bigrams = list(bigrams(wordlist_tokens))
    
    # Creating an empty list to store each bigram prob
    prob_list = []
    
    # Looping through the bigram list 
    for bi_gram in wordlist_bigrams:
        previous = bi_gram[0]
        current = bi_gram[1]
        
        # extracting bigram freq from the table
        bigram_freq = table.loc[previous][current]
        
        # Converting previous word into unigram
        unigram = tuple(previous.split())
        # extracting the unigram frequency from unigram frequency in the corpus
        unigram_freq = frequienceU.get(unigram)
        
        # if unigram is not availible in the corpus we will assign it to 1, tp avoid Nonetype devision
        # Note assigning it to wont effect the probablity of the sentance
        if unigram_freq is None:
            unigram_freq = 1 
        
        # Calculating the current bigram prob in the corpus
        
        bigram_prob = bigram_freq/unigram_freq
        
        prob_list.append(bigram_prob)
    sentance_prob = np.prod(prob_list)
    return sentance_prob
    

In [18]:
test_1 = 'this is the'
calculate_sent_prob(test_1)

0.0013972334777141262

Data in the corpus is less thats why most of the values are zero

### Worng Words

In [19]:
# Will result bigrams which has zero probablity in the prob table
def incorrect_bigrams(sent):
    incorrect_bigrms = []
    prob_table = bigram_prob_table(sent)
    tokens = word_tokenize(sent)
    bi_grams = list(bigrams(tokens))
    for bi in bi_grams:
        if prob_table.loc[bi[0]][bi[1]] == 0:
            incorrect_bigrms.append(bi)
    
    return incorrect_bigrms

# will return those words which are not in the corpus or which are not a unigram
def miss_spelled_words(sent):
    inocrrect_unigrms = []
    prob_table = bigram_prob_table(sent)
    tokens = word_tokenize(sent)
    uni_grams = list(ngrams(tokens,1))
    for uni in uni_grams:
        if uni not in frequienceU.keys():
            inocrrect_unigrms.append(uni[0])
    return inocrrect_unigrms

In [20]:
test_3 = 'you is a boys'
incorrect_bigrams(test_3)

[('you', 'is'), ('a', 'boys')]

In [21]:
test_4 = "which are tha spelld wrds in the test"
miss_spelled_words(test_4)

['tha', 'spelld', 'wrds']

### Sugestions

In [22]:
def sugestions(sent):
    incorrect_bigrms = incorrect_bigrams(sent)
    for ic_bigram in incorrect_bigrms:
        bigram_dic = {}
        for key,value in frequienceB.items():
            if key[0] == ic_bigram[0]:
                bigram_dic[key] = value
            
        # Sorting the dic in Desending order , first key will have highest frequency
        sorted_bigrams = dict( sorted(bigram_dic.items(), key=operator.itemgetter(1),reverse=True))
    
        # slicing only the first three elements from the dictionary, which are top three most occuring 
        sugestion = dict(itertools.islice(sorted_bigrams.items(), 3))
    
        # printing sugestions
        print(f'Suggested words for {ic_bigram} : {sugestion}')

In [23]:
test_5 = 'we is testing this in'
sugestions(test_5)

Suggested words for ('we', 'is') : {('we', 'read'): 118, ('we', 'can'): 35, ('we', 'begin'): 30}
Suggested words for ('is', 'testing') : {('is', 'advised.Fr.'): 343, ('is', 'advised.As'): 57, ('is', 'advised.In'): 35}
Suggested words for ('testing', 'this') : {}


### Replacing Words

In [24]:
def replacing_words(sent):
    # List of incorrect bigrams
    incorrect_bigrms = incorrect_bigrams(sent)
    # If the list is not empty
    while len(incorrect_bigrms)>0:
        # Initializing an empty dictionary
        bigram_dic = {}
        
        for key,value in frequienceB.items():
            if key[0] == incorrect_bigrms[0][0]:
                bigram_dic[key] = value
            
        # Sorting in Desending order , first bigram will have highest frequency
        sorted_bigrams = sorted(bigram_dic.items(), key=operator.itemgetter(1),reverse=True)
        
        wrong_word = incorrect_bigrms[0][1]
        correct_word = sorted_bigrams[0][0][1]
        
        sent  = sent.replace(wrong_word,correct_word)
        
        incorrect_bigrms = incorrect_bigrams(sent)
    return sent
        

There can also be other problems which i dont know. but one problem i have noticed in my code is.
If the first word of the input sentance is not in the corpus then this `replacing_words` function will through an error ... Because `bigram_dic` in the function will be empty and `sorted_bigrams` variable in the function block will also be empty and this line of code `correct_word = sorted_bigrams[0][0][1]` will try to do indexing in empty list which will trough error.
I tried but due to my inexperience i could not handle this issue

In [25]:
test_3

'you is a boys'

In [26]:
replacing_words(test_3)

'you to a life'